# Scraping data from Retrosheet.org: Part 2 - parsing the split page

In [18]:
from bs4 import BeautifulSoup;
import requests;
import re;
import pandas as pd;
import numpy as np;

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; \
    Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'};
retro_page_affix = "https://www.retrosheet.org/boxesetc/";


In [19]:
df_player_career_stats = pd.read_csv("player_career_split_url.csv", usecols = ["ID", "Name", "split_url"]);
df_player_career_stats

,ID,Name,split_url
0,0,David Aardsma,https://www.retrosheet.org/boxesetc/A/Jaardd00...
1,1,Hank Aaron,https://www.retrosheet.org/boxesetc/A/Jaaroh10...
2,2,Tommie Aaron,https://www.retrosheet.org/boxesetc/A/Jaarot10...
3,3,Don Aase,https://www.retrosheet.org/boxesetc/A/Jaased00...
4,4,Andy Abad,https://www.retrosheet.org/boxesetc/A/Jabada00...
...,...,...,...
16004,19913,Bob Zupcic,https://www.retrosheet.org/boxesetc/Z/Jzupcb00...
16005,19914,Frank Zupo,https://www.retrosheet.org/boxesetc/Z/Jzupof10...
16006,19915,Paul Zuvella,https://www.retrosheet.org/boxesetc/Z/Jzuvep00...
16007,19916,George Zuverink,https://www.retrosheet.org/boxesetc/Z/Jzuveg10...


In [12]:
# Process if have chance. Have a larger chance to be blocked by Retrosheet.
df_player_yearly_stats = pd.read_csv("player_season_split_url.csv", usecols = ["ID", "Name", "Season", "Team", "split_url"]);

In [20]:
url = df_player_career_stats.loc[1,"split_url"];
url

response = requests.get(url, headers=headers);
text = BeautifulSoup(response.text, 'html.parser');

if response.status_code != 200:
    raise Exception(f"The status code is not 200! It is {response.status_code}.");

In [38]:
pret = text.findAll("pre");

for ta in pret:
    if ta.get_text().find("Total") != -1:
        print("Found batting record chunk");
        break;

ta.contents[0].splitlines()

Found batting record chunk


'Away  1651  6396 1094 1960 341  61 370 1180  657 134  729  15   6  63   0 123 197  123  41  .306  .369  .552'

1.0